In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder

<h1>Linear Damage</h1>

In [3]:
df = pd.read_csv('input/results_complete_linear.csv')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['damage'], axis=1), df['damage'], test_size=0.10, random_state=41)

In [8]:
scaler = StandardScaler()
hot_encoder = OneHotEncoder(sparse=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
%%time
path = 'models/linear_reg'
if (os.path.exists(path)):
    model = keras.models.load_model(path)
else:
    model = keras.Sequential()
    model.add(keras.Input(shape=(127)))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal())) #80
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(1, activation=tf.keras.layers.LeakyReLU(alpha=0.2)))
    model.summary()
    model.compile(loss='huber', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=['mse', 'mae'])
    history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test)) #1000 epocas
    model.save(path)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 80)                10240     
                                                                 
 dense_17 (Dense)            (None, 80)                6480      
                                                                 
 dense_18 (Dense)            (None, 80)                6480      
                                                                 
 dense_19 (Dense)            (None, 1)                 81        
                                                                 
Total params: 23,281
Trainable params: 23,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
726/726 [==============================] - 1s 1ms/step - loss: 0.1046 - mse: 0.2846 - mae: 0.3360 - val_loss: 0.0602 - val_mse: 0.1535 - val_mae: 0.2606
Epoch 2/1000
726

INFO:tensorflow:Assets written to: models/linear_reg/assets


INFO:tensorflow:Assets written to: models/linear_reg/assets


CPU times: user 16min 11s, sys: 1min 5s, total: 17min 17s
Wall time: 10min 54s


In [16]:
pd.DataFrame(history.history).to_csv('output/history_mlp_linear')

loss       mse       mae  val_loss   val_mse   val_mae
0    0.104597  0.284615  0.336003  0.060174  0.153456  0.260579
1    0.050684  0.108195  0.243062  0.045431  0.110223  0.233900
2    0.041340  0.084375  0.224335  0.040359  0.096199  0.221267
3    0.037434  0.076321  0.214405  0.037256  0.085629  0.213611
4    0.033696  0.068271  0.205412  0.033782  0.073332  0.204957
..        ...       ...       ...       ...       ...       ...
995  0.006611  0.014299  0.073472  0.007268  0.014634  0.074756
996  0.006235  0.012524  0.072363  0.007810  0.015947  0.076709
997  0.006657  0.013423  0.073632  0.008242  0.016981  0.077017
998  0.006311  0.012634  0.072714  0.010534  0.036930  0.080908
999  0.006470  0.014140  0.072840  0.006890  0.013866  0.074145

[1000 rows x 6 columns]

In [15]:
plt.plot(history.history['loss'])
plt.plot(history.history['mse'])
plt.plot(history.history['mae'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_mse'])
plt.plot(history.history['val_mae'])
plt.show()

In [22]:
pred = model.predict(X_test)

81/81 [==============================] - 0s 757us/step


<h3>Classification Report</h3>

In [23]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       0.94      0.94      0.94        18
           2       1.00      0.96      0.98        23
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        23
           5       1.00      0.65      0.79        20
           6       1.00      0.96      0.98        23
           7       1.00      1.00      1.00        25
           8       1.00      0.96      0.98        25
           9       1.00      1.00      1.00        13
          10       0.72      0.91      0.81        23
          11       1.00      0.94      0.97        17
          12       1.00      0.88      0.93        16
          13       0.94      1.00      0.97        31
          14       1.00      1.00      1.00        30
          15       1.00      1.00      1.00        19
          16       1.00      1.00      1.00        25
          17       1.00    

<h1>Exponential Damage</h1>

In [24]:
df_exp = pd.read_csv('input\\results_complete_exponential.csv')

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_exp.drop(['elem_damaged', 'damage'], axis=1), df_exp['elem_damaged'], test_size=0.10, random_state=41)

In [26]:
scaler = StandardScaler()
hot_encoder = OneHotEncoder(sparse=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = hot_encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test = hot_encoder.transform(y_test.values.reshape(-1, 1))

In [27]:
%%time
path = 'models\\exp_class'
if (os.path.exists(path)):
    model = keras.models.load_model(path)
else:
    model = keras.Sequential()
    model.add(keras.Input(shape=(126)))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal())) #80
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(129, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test)) #1000 epocas
    model.save(path)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                10160     
                                                                 
 dense_9 (Dense)             (None, 80)                6480      
                                                                 
 dense_10 (Dense)            (None, 80)                6480      
                                                                 
 dense_11 (Dense)            (None, 129)               10449     
                                                                 
Total params: 33,569
Trainable params: 33,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
726/726 [==============================] - 1s 1ms/step - loss: 4.9790 - accuracy: 0.0183 - val_loss: 4.8442 - val_accuracy: 0.0167
Epoch 2/1000
726/726 [================

INFO:tensorflow:Assets written to: models\exp\assets


INFO:tensorflow:Assets written to: models\exp\assets


Wall time: 14min 29s


In [29]:
pred = model.predict(X_test)

81/81 [==============================] - 0s 810us/step


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

<h3>Classification Report</h3>

In [31]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       1.00      0.94      0.97        18
           2       1.00      0.83      0.90        23
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        23
           5       0.55      0.30      0.39        20
           6       1.00      0.96      0.98        23
           7       0.89      1.00      0.94        25
           8       1.00      0.96      0.98        25
           9       1.00      1.00      1.00        13
          10       0.40      0.96      0.56        23
          11       1.00      0.94      0.97        17
          12       1.00      0.81      0.90        16
          13       1.00      1.00      1.00        31
          14       1.00      0.97      0.98        30
          15       1.00      1.00      1.00        19
          16       1.00      1.00      1.00        25
          17       1.00    

C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

<h1>Sigmoid-like Damage</h1>

In [32]:
df_sig = pd.read_csv('input\\results_complete_sigmoid_like.csv')

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df_sig.drop(['elem_damaged', 'damage'], axis=1), df_sig['elem_damaged'], test_size=0.10, random_state=41)

In [34]:
scaler = StandardScaler()
hot_encoder = OneHotEncoder(sparse=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = hot_encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test = hot_encoder.transform(y_test.values.reshape(-1, 1))

In [35]:
%%time
path = 'models\\sig_class'
if (os.path.exists(path)):
    model = keras.models.load_model(path)
else:
    model = keras.Sequential()
    model.add(keras.Input(shape=(126)))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal())) #80
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(80, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(keras.layers.Dense(129, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test)) #1000 epocas
    model.save(path)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                10160     
                                                                 
 dense_13 (Dense)            (None, 80)                6480      
                                                                 
 dense_14 (Dense)            (None, 80)                6480      
                                                                 
 dense_15 (Dense)            (None, 129)               10449     
                                                                 
Total params: 33,569
Trainable params: 33,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
726/726 [==============================] - 1s 1ms/step - loss: 4.9334 - accuracy: 0.0049 - val_loss: 4.9141 - val_accuracy: 0.0039
Epoch 2/1000
726/726 [================

INFO:tensorflow:Assets written to: models\sig\assets


INFO:tensorflow:Assets written to: models\sig\assets


Wall time: 14min 10s


In [38]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [36]:
pred = model.predict(X_test)

81/81 [==============================] - 0s 778us/step


<h3>Classification Report</h3>

In [37]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.45      0.81      0.58        21
           1       0.93      0.72      0.81        18
           2       0.95      0.78      0.86        23
           3       1.00      0.80      0.89        20
           4       0.90      0.83      0.86        23
           5       0.09      0.95      0.17        20
           6       0.77      0.74      0.76        23
           7       1.00      0.76      0.86        25
           8       1.00      0.84      0.91        25
           9       0.91      0.77      0.83        13
          10       0.50      0.04      0.08        23
          11       0.93      0.82      0.87        17
          12       1.00      0.62      0.77        16
          13       1.00      0.42      0.59        31
          14       1.00      0.57      0.72        30
          15       1.00      0.79      0.88        19
          16       0.95      0.80      0.87        25
          17       0.93    

C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Gabriel\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu